In [1]:
%run 00.style.ipynb

/Users/piazza/Playground/tweet-in-love


In [2]:
import spacy
from spacy.tokens import DocBin

from spacytextblob.spacytextblob import SpacyTextBlob
from sklearn.preprocessing import LabelBinarizer
from sklearn.base import TransformerMixin
from typing import List, Tuple, Iterable

from tweet_in_love.settings import GlobalSettings, ModelSettings
from tweet_in_love.preprocessor import PreProcessor

settings = GlobalSettings()

In [3]:
settings.train_csv_path

PosixPath('data/train/train.csv')

# Load data

In [4]:
df_train = pd.read_csv(settings.train_csv_path)
df_test = pd.read_csv(settings.test_csv_path)

In [5]:
df_train.sample(5)

,content,sentiment
22038,My computer died,sadness
8868,"I am sooooo happy! Finally, Kean Cipriano repl...",love
1092,"gooooooodmorning world(: god bless, and have a...",worry
6631,@emmarler i am jealous of your mom talking to ...,worry
25462,@lewisking don't you procrastinate! (like I ...,worry


In [6]:
# Download small model pretrained on web data
# !spacy download en_core_web_sm

In [7]:
nlp = spacy.load("en_core_web_sm")

In [8]:
df_train.head()

,content,sentiment
0,NEW YORK DOLLS TOMORROW NIGHT @ THE DEPOT CANC...,fun
1,@cosRobPerkins You're the third person to ask ...,surprise
2,"@thomaskattus you asked about my SF schedule, ...",neutral
3,SWAY SWAY BABY zommgg love it need more screa...,happiness
4,@LaDyBuG21 Thanks,neutral


# Preprocess

In [15]:
preprocessor = PreProcessor(
    nlp=nlp,
    classes=df_train.sentiment,
)

In [10]:
train_docs = preprocessor.df_to_doc(df=df_train)

In [11]:
test_docs = preprocessor.df_to_doc(df=df_test)

[autoreload of tweet_in_love.preprocess failed: Traceback (most recent call last):
  File "/Users/piazza/Playground/tweet-in-love/.venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/piazza/Playground/tweet-in-love/.venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/piazza/Playground/tweet-in-love/.venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/Users/piazza/Playground/tweet-in-love/.venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 317, in update_class
    update_instances(old, new)
  File "/Users/piazza/Playground/tweet-in-love/.venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 280, in update_instances
    ref.__class__ = new
  File "pydantic/main.py", line 422, in pydantic.main.B

In [16]:
preprocessor.save_docs(train_docs, test_docs, tag="web_sm")

(PosixPath('data/train/train_web_sm.spacy'),
 PosixPath('data/test/test_web_sm.spacy'))

----
# Take a look at the preprocessed data

In [17]:
sample = preprocessor.df_to_doc(df=df_train.sample(10))

In [18]:
for doc in sample:
    print(doc)
    print(doc.ents)
    print([(token.lemma_) for token in doc])

    print([(token.vector_norm) for token in doc])
    print()

my girlies @VeeVeeBOMBSHELL &amp; @ReeseCromwell r leavin me today  Vanessa, I'll c u when u'r black *I mean back and Reese, I'll c u Sunday
(girlies @VeeVeeBOMBSHELL &, @ReeseCromwell, today, Reese, Sunday)
['my', 'girlie', '@VeeVeeBOMBSHELL', '&', 'amp', ';', '@ReeseCromwell', 'r', 'leavin', 'I', 'today', ' ', 'Vanessa', ',', 'I', "'ll", 'c', 'u', 'when', "u'r", 'black', '*', 'I', 'mean', 'back', 'and', 'Reese', ',', 'I', "'ll", 'c', 'u', 'Sunday']
[9.148404, 6.9401517, 8.537192, 9.016395, 7.8125644, 9.812474, 7.7398686, 6.605575, 6.785845, 7.818848, 9.122936, 8.097483, 7.952559, 7.6849937, 7.36979, 7.771745, 7.294192, 6.14944, 9.8892975, 6.5152082, 6.4241886, 7.3274245, 8.844927, 7.944716, 6.912108, 9.016117, 7.1334057, 7.213392, 6.9743543, 7.7682514, 7.029519, 7.044068, 9.256737]

Good Morning Sunshines!
(Good Morning Sunshines,)
['Good', 'Morning', 'Sunshines', '!']
[7.8765597, 7.4672213, 6.9710283, 10.281048]

urgh, over slept for work, still done no revision and im SO snappy tod

- The vanilla pretrained model is already able to detect `@mentions`.
  It also calculates their vector.
- `#hashtags` are treated as words, removing the `#`

# Add a sentiment score

In [19]:
nlp2 = spacy.load("en_core_web_sm")

# Add spaCyTextBlob-based sentiment analysis
nlp2.add_pipe("spacytextblob")

In [20]:
preprocessor = PreProcessor(
    nlp=nlp2,
    classes=df_train.sentiment,
)

In [21]:
sample = preprocessor.df_to_doc(df=df_train.sample(10))

In [23]:
for doc in sample:
    print(doc)
    print(doc.ents)
    print(
        f"pola:{doc._.polarity} subj:{doc._.subjectivity}\nassess:{doc._.assessments}"
    )
    print("..")
    print([(token.lemma_) for token in doc])
    print([(token.vector_norm) for token in doc])

    print("--\n")

bought beer today
(today,)
pola:0.0 subj:0.0
assess:[]
..
['buy', 'beer', 'today']
[8.238936, 7.4964523, 10.483919]
--

@honytawk why do we need to wait another year for toy story 3?!
(another year, 3)
pola:0.0 subj:0.0
assess:[]
..
['@honytawk', 'why', 'do', 'we', 'need', 'to', 'wait', 'another', 'year', 'for', 'toy', 'story', '3', '?', '!']
[6.6258135, 8.821145, 9.228143, 9.22015, 7.450351, 8.512473, 7.317643, 8.592593, 6.8815374, 6.471072, 6.466468, 6.1588793, 8.160021, 9.446849, 10.280108]
--

@torriRAWR OMG apparantley green day are touring here in december..we are so going
(green day, december)
pola:-0.2 subj:0.3
assess:[(['green'], -0.2, 0.3, None)]
..
['@torriRAWR', 'OMG', 'apparantley', 'green', 'day', 'be', 'tour', 'here', 'in', 'december', '..', 'we', 'be', 'so', 'go']
[6.716484, 8.642971, 6.7387495, 7.5328865, 7.077684, 8.45785, 7.413875, 8.412568, 8.031971, 9.421901, 7.8196898, 7.830092, 7.292568, 8.738072, 7.3948]
--

&quot;To boldly go where no one has gone before...&quo

- Vanilla TextBlob is already providing some nice data points to build a baseline classifier

In [24]:
train_docs = preprocessor.df_to_doc(df=df_train)

In [25]:
test_docs = preprocessor.df_to_doc(df=df_test)

In [26]:
preprocessor.save_docs(train_docs, test_docs, tag="txtblob_sm")

(PosixPath('data/train/train_txtblob_sm.spacy'),
 PosixPath('data/test/test_txtblob_sm.spacy'))